In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import csv
import regex as re

In [ ]:
df = pd.read_csv("Sales_English(basic clean).csv")

In [ ]:
df = df.rename(columns=str.lower).rename(columns={'most recent activity': 'most recent activity date'})
df.head()

,customer type,sales representative,province,city,most recent activity date,most recent sales representative,customer pool,data creator,claim date,recent change date,recent editor,status,belonging department,business registration,do-not-disturb,number of signed contract,amount received,total contract amount,account receivable,non-collectibles,amount to be collected,actual amount collected,effective sales amount,total sales amount,finished customer,customer rating,interest category,customer source,source secondary classification,source third classification,source fourth classification,business category,interest program,information date,customer source star rating,dududu? (yes/no),source code,customer update status,percentage,program sales total,expected sales,number of calls,repeated? (yes/no),repeated date,most recent publish date
0,Individual,Weijingjuan,unknown,unknown,2021-09-15,Weijingjuan,Entire - Collectible,Zhangmeng,2021-09-15,2021-10-09 15:08,Weijingjuan,Collected,Southern 2,Yes,False,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,40,D,Online Operations,Media,Jinritoutiao,Form Data,Employed,ENPC-DBA-IMM,2021-09-15,***,False,C,1,0.05,588000,29400.00,3,unknown,unknown,2021-09-15 16:51
1,Individual,Lishanshan,unknown,unknown,2021-10-03,Lishanshan,Entire,Hemengjun,2021-01-16,2021-10-03 16:42,Lishanshan,Collected,129shi,Yes,False,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,40,D,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,ENPC-DBA-IMM,2021-10-03,***,False,C,1,0.05,588000,29400.00,1,Yes,2021-01-06,2021-10-03 16:42
2,Individual,Hewei,unknown,unknown,2021-04-13,Hewei,Entire,Hemengjun,2021-01-16,2021-08-20 10:27,Administrator,Collected,27jun,Yes,False,1,2000.0,370000.0,0.0,0.0,0.0,2000.0,1,2000.0,1,95,F,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,UCA-DBA,2021-04-13,***,False,C,unknown,0.00,268000,0,0,unknown,unknown,unknown
3,Individual,Lishanshan,unknown,unknown,2021-08-16,Lishanshan,Entire,Hemengjun,2021-01-16,2021-08-16 17:06,Lishanshan,Collected,129shi,Yes,False,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,40,D,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,ENPC-DBA-IMM,2021-08-16,***,False,C,1,0.05,588000,29400.00,2,Yes,2021-02-03,2021-08-16 17:06
4,Individual,Zhouwei,unknown,unknown,2021-09-24,Zhouwei,Entire,Hemengjun,unknown,2021-09-24 15:07,Zhouwei,Self-Configured,Mount Everest,Yes,False,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,40,D,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,General DBA,2021-09-24,**,True,C,1,0.05,300000,15000.00,5,unknown,unknown,2021-09-24 15:07


In [ ]:
#For ongoing customers, Total Contract Amount is greater than 0 and Amount Recieved might be 0. 
#For past customers, Amount Received is greater than 0 and Total Contract Amount is 0.
successful_customers = df[(df['amount received'] > 0) | (df['total contract amount'] > 0)]
successful_customers.head()

,customer type,sales representative,province,city,most recent activity date,most recent sales representative,customer pool,data creator,claim date,recent change date,recent editor,status,belonging department,business registration,do-not-disturb,number of signed contract,amount received,total contract amount,account receivable,non-collectibles,amount to be collected,actual amount collected,effective sales amount,total sales amount,finished customer,customer rating,interest category,customer source,source secondary classification,source third classification,source fourth classification,business category,interest program,information date,customer source star rating,dududu? (yes/no),source code,customer update status,percentage,program sales total,expected sales,number of calls,repeated? (yes/no),repeated date,most recent publish date
2,Individual,Hewei,unknown,unknown,2021-04-13,Hewei,Entire,Hemengjun,2021-01-16,2021-08-20 10:27,Administrator,Collected,27jun,Yes,False,1,2000.0,370000.0,0.0,0.0,0.0,2000.0,1,2000.0,1,95,F,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,UCA-DBA,2021-04-13,***,False,C,unknown,0.0,268000,0,0,unknown,unknown,unknown
14,Individual,Gexiaodan,unknown,unknown,2021-03-09,Zhangmai,Entire,Zhangping,unknown,2021-08-20 10:27,Administrator,Self-Configured,Teaching Center,Yes,False,1,2000.0,2000.0,201319.9,0.0,201319.9,203319.9,2,203319.9,1,100,Official Student,Channel Center,Enrollment Proxy,unknown,unknown,Employed,ISTEC-DBA,2021-03-09,**,False,B,unknown,0.0,178000,0,0,unknown,unknown,unknown
32,Individual,Morong,unknown,unknown,2021-01-04,Morong,South - 2,Libin,unknown,2021-08-20 10:27,Administrator,Self-Configured,Southern 2,No,False,0,114654.4,0.0,0.0,0.0,0.0,114654.4,2,114654.4,1,95,F,Channel Center,Enrollment Proxy,unknown,unknown,Employed,ISTEC-DEM,2021-01-04,**,False,B,unknown,0.0,178000,0,0,unknown,unknown,unknown
43,Individual,Gexiaodan,unknown,unknown,2021-01-07,Shilinyu,Entire,Hemengjun,unknown,2021-08-20 10:27,Administrator,Self-Configured,Teaching Center,Yes,False,0,161942.4,0.0,0.0,0.0,0.0,161942.4,2,161942.4,1,95,F,Online Operations,Media,Wechat - Pengyouquan,unknown,Employed,UM-MHM,2021-01-07,**,False,C,unknown,0.0,156000,0,0,unknown,unknown,unknown
78,Individual,Morong,unknown,unknown,2021-02-24,Morong,South - 2,Libin,unknown,2021-08-20 10:27,Administrator,Signed Contract,Southern 2,Yes,False,0,152000.0,0.0,67158.5,0.0,67158.5,219158.5,2,219158.5,1,100,Official Student,Channel Center,Enrollment Proxy,unknown,unknown,Employed,ISTEC-DEM,2021-02-24,**,False,B,unknown,0.0,178000,0,0,unknown,unknown,unknown


In [ ]:
num_suc_sales = successful_customers.groupby("sales representative").count()['city'].sort_values(ascending = False)
num_suc_sales.head()

sales representative
Gexiaodan     75
Lihuanhuan    34
Zhaokeqing    19
Fengyanyan    15
Lirui         11
Name: city, dtype: int64

In [ ]:
tot_assigned_sales = df.groupby(['sales representative']).count()['customer type']
tot_assigned_sales.head()

sales representative
Administrator    519
Chenchaohui       29
Chendongmei       41
Chenhaotian        8
Chenlimin        218
Name: customer type, dtype: int64

In [ ]:
sales_perf = num_suc_sales.to_frame().join(tot_assigned_sales).rename({'customer type': '# of assigned customer', 'city': '# of successful customer'}, axis='columns').fillna(0)
sales_perf['succ/assigned'] = sales_perf['# of successful customer']/sales_perf['# of assigned customer']
sales_perf.sort_values(by = 'succ/assigned', ascending = False).head(10)

,# of successful customer,# of assigned customer,succ/assigned
sales representative,,,
Shizhanbin,1,1,1.000000
Gexiaodan,75,106,0.707547
Yangtiequan,2,3,0.666667
Liwen,4,19,0.210526
Lihuanhuan,34,170,0.200000
Mali,2,19,0.105263
Chendongmei,4,41,0.097561
Huangwanyun,3,39,0.076923
Tanxiaoxin,4,54,0.074074


In [ ]:
successful_customers[['amount received','total contract amount','number of signed contract']]

,amount received,total contract amount,number of signed contract
2,2000.00,370000.0,1
14,2000.00,2000.0,1
32,114654.40,0.0,0
43,161942.40,0.0,0
78,152000.00,0.0,0
...,...,...,...
8871,2000.00,0.0,0
9129,2000.00,0.0,0
9530,2000.00,0.0,0
10114,2000.00,0.0,0


In [ ]:
df['success'] = (df['amount received'] > 0) | (df['total contract amount'] > 0)
#df.to_csv(r'Sales_cleaning_sucess.csv', index=False)

In [ ]:
df.columns

Index(['customer type', 'sales representative', 'province', 'city',
       'most recent activity date', 'most recent sales representative',
       'customer pool', 'data creator', 'claim date', 'recent change date',
       'recent editor', 'status', 'belonging department',
       'business registration', 'do-not-disturb', 'number of signed contract',
       'amount received', 'total contract amount', 'account receivable',
       'non-collectibles', 'amount to be collected', 'actual amount collected',
       'effective sales amount', 'total sales amount', 'finished customer',
       'customer rating', 'interest category', 'customer source',
       'source secondary classification', 'source third classification',
       'source fourth classification', 'business category', 'interest program',
       'information date', 'customer source star rating', 'dududu? (yes/no)',
       'source code', 'customer update status', 'percentage',
       'program sales total', 'expected sales', 'number of 

In [ ]:
for column in df.select_dtypes(exclude=['int64']):
    print(column, df[column].unique())

customer type ['Individual' 'Channel' 'Education Abroad' 'Corporate' 'C+B']
sales representative ['Weijingjuan' 'Lishanshan' 'Hewei' 'Zhouwei' 'Lirui' 'Lina' 'Shilinyu'
 'Daiminmin' 'Leiduanyu' 'Zhangmai' 'Gexiaodan' 'Xuyan' 'Tangyiqiong'
 'Wengxiaowen' 'Wanglanmei' 'Maobeilei' 'Xiebingbing' 'Linna - Resign'
 'Yanying' 'Fengyanyan' 'Libin' 'Morong' 'Zhangping' 'Lihuanhuan'
 'Chenchaohui' 'Lidexiang' 'Hulan' 'Xujun' 'Chenlimin' 'Sunningning'
 'Hutianhong' 'Qianshujia' 'Huanglidi' 'Fuguoliang' 'Dujia' 'Wangyifan'
 'Dingzhenhua' 'Xuyanli' 'Gonglijun' 'Xiebin' 'Yurong' 'Zhaokeqing'
 'Jiangjin' 'Zhuyong' 'Yangtangzan' 'Wangjian' 'Mawanli' 'Libo' 'Raoyizhi'
 'Mashuya' 'Zhangyao' 'Shengboyang' 'Fuyu' 'Gujiong' 'Wanglili' 'Wanghong'
 'Liwen' 'Huangwanyun' 'Zhanghao' 'Administrator' 'Donghaosai'
 'Chendongmei' 'Jiajing' 'Jinning' 'Xiaodong' 'Yangtiequan' 'Honghaixia'
 'Shizhanbin' 'Zhouhualin' 'Wumingzhu' 'Yangzhan' 'Luosiyi' 'Chenhaotian'
 'Zhouhonglin' 'Mali' 'Wucheng' 'Youmiao' 'Quwenpeng' '

In [ ]:
#find numerical columns & categorical columns
sales_cols = [column for column in df.select_dtypes(include='number')][:-2]
print("Sales columns", sales_cols)


Sales columns ['number of signed contract', 'amount received', 'total contract amount', 'account receivable', 'non-collectibles', 'amount to be collected', 'actual amount collected', 'effective sales amount', 'total sales amount', 'finished customer']


In [ ]:
#Find datetime columns in data
date_cols = df.rename(columns=str.lower).filter(regex=(".*(date)+.*")).drop(['customer update status'], axis=1).columns
print("Datetime columns", date_cols)

Datetime columns Index(['most recent activity date', 'claim date', 'recent change date',
       'information date', 'repeated date', 'most recent publish date'],
      dtype='object')


In [ ]:
date_sales_cols = list(sales_cols)+list(date_cols)
cat_cols = [column for column in df.drop(columns = date_sales_cols ,axis=1)]
cat_cols

['customer type',
 'sales representative',
 'province',
 'city',
 'most recent sales representative',
 'customer pool',
 'data creator',
 'recent editor',
 'status',
 'belonging department',
 'business registration',
 'do-not-disturb',
 'customer rating',
 'interest category',
 'customer source',
 'source secondary classification',
 'source third classification',
 'source fourth classification',
 'business category',
 'interest program',
 'customer source star rating',
 'dududu? (yes/no)',
 'source code',
 'customer update status',
 'percentage',
 'program sales total',
 'expected sales',
 'number of calls',
 'repeated? (yes/no)',
 'success']

In [ ]:
#Change program sales to numeric
df['program sales total'] = df['program sales total'].map(lambda x: x.lstrip('\t')).replace('unknown',0).astype(int)
df['program sales total'].head()

0    588000
1    588000
2    268000
3    588000
4    300000
Name: program sales total, dtype: int64

In [ ]:
df['interest category'].unique()

array(['D', 'F', 'B', 'C', 'G', 'Official Student', 'W', 'A', 'I', 'E',
       'X', 'J', 'unknown', 'H', 'R'], dtype=object)

In [ ]:
#F-paid interview fee, A - 1/week, B - 1/2 weeks, C - 1/3 weeks, D - 1/2 months 
#E - EMC acitivity interest/No particular interest, #G - 未接, X - 错号, H - 空号, 
#Official Student, #W - 有兴趣没对应项目
#Official Student - 100, F - 80, A - 60, B - 50, C - 40, D -25, W - 15, E,G - 10, X,H,Unkown - 0, 
num_int = df['interest category'].replace({'J':'I','R':'I','Official Student':100, 'F':80, 'A':60, 'B':50, 'C':40, 'D':25,
                                'W':15, 'E':10, 'G':10, 'W':10, 'I':10, 'X':0, 'H':0, 'unknown':0}).replace('I',10)

In [ ]:
r = np.corrcoef(df['customer rating'], num_int)
r
#Highly correlated, drop interest category
cat_cols.remove('interest category')

In [ ]:
cat_cols = [cols for cols in cat_cols if cols != 'interest category' and cols != 'percentage' and cols != 'success']
cat_cols

['customer type',
 'sales representative',
 'province',
 'city',
 'most recent sales representative',
 'customer pool',
 'data creator',
 'recent editor',
 'status',
 'belonging department',
 'business registration',
 'do-not-disturb',
 'customer rating',
 'customer source',
 'source secondary classification',
 'source third classification',
 'source fourth classification',
 'business category',
 'interest program',
 'customer source star rating',
 'dududu? (yes/no)',
 'source code',
 'customer update status',
 'program sales total',
 'expected sales',
 'number of calls',
 'repeated? (yes/no)']

In [ ]:
df['number of calls'] = df['number of calls'].replace('unknown',0).astype(int)
df['dududu? (yes/no)'] = df['dududu? (yes/no)'].astype('int')
df['repeated? (yes/no)'] = df['repeated? (yes/no)'].replace({'Yes':1,'unknown':0})

In [ ]:
X = df[cat_cols]
y = df['success']

cnt_int_prog = pd.DataFrame(X.groupby(['interest program']).count().sort_values(by ='number of calls', ascending = False)['customer type'])
cnt_int_prog.index

Index(['General DBA', 'Intelligent Manufacturing DBA', 'Health DBA', 'B-Proxy',
       'General MBA', 'Health MBA', 'Online Micro Lessons', 'unknown',
       'Education DBA', 'Finance DBA', 'UM-EMBA', 'Data Tranformation DBA',
       'Data Econ DBA', 'UM-DHM', 'Study Abroad Phd', 'Others', 'UM-EDBA',
       'ISTEC-EMBA (City Partner Price)', 'Asset Management DBA', 'ISTEC-DBA',
       'UCA-DBA', 'ISTEC-DBA-IMM', 'ISTEC-MBA', 'ISTEC-DEM', 'UM-MHM',
       'Intelligent Manufacturing MBA', 'ENPC-DBA-IMM', 'Data Science DBA',
       'Montepellier - EMBA', 'NEOMA-DBA', 'Study Abroad - Bus',
       'ISTEC-MBA-IMM', 'NEOMA-DDE', 'DIU-MBA HCM', 'ISTEC-MBA-OL',
       'ISTEC-DDT', 'ISTEC-FDBA', 'NEOMA-DAM', 'ENPC-DIMM', 'Culture DBA',
       'ENPC-DHM', 'Study Abroad Master', 'Wealth Management DBA',
       'ISTEC-DBA-OL', 'B-Partner', 'Data Marketing DBA', 'Online MBA',
       'UM-PHD(Study Abroad, Require pre-PHD)', 'UM-PHD', 'ESC-BBA-OL',
       'Online DBA', 'NEOMA-DDM', 'NEOMA-FDBA', 'Fina

In [ ]:
#generalize program by degree
def generalizeprogram(column):
    for i in range(len(column)):
        if 'FDBA' in column[i]:
            column[i] = 'FDBA'
        elif 'DBA' in column[i]:
            column[i] = 'DBA'
        elif 'FMBA' in column[i]:
            column[i] = 'FMBA'
        elif 'EMBA' in column[i]:
            column[i] = 'EMBA'
        elif 'MBA' in column[i]:
            column[i] = 'MBA'
        elif 'Online' in column[i]:
            column[i] = 'Online'
        elif 'IUKL' in column[i]:
            column[i] = 'IUKL'
        elif 'Study Abroad' in column[i]:
            column[i] = 'Study Abroad'
        elif 'B-' in column[i]:
            column[i] = 'B'
        elif 'NEOMA' in column[i]:
            column[i] = 'NEOMA'
        elif 'unknown' in column[i]:
            column[i] = 'unknown'
        else:
            column[i] = 'Others'

X['interest program'] = generalizeprogram(list(X['interest program']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Train Test split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics

X_dum = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.3, random_state=1)

Model Selection

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("Train accuracy: ", model.score(X_train,y_train))
print("Test accuracy: ", model.score(X_test,y_test))
print(classification_report(y_test,pred))

Train accuracy:  0.9705921139390743
Test accuracy:  0.968
              precision    recall  f1-score   support

       False       0.98      0.99      0.98      3147
        True       0.49      0.29      0.37       103

    accuracy                           0.97      3250
   macro avg       0.73      0.64      0.67      3250
weighted avg       0.96      0.97      0.96      3250



In [ ]:
# The Decision tree Classifier
from sklearn.tree import DecisionTreeClassifier 
# Create Decision Tree classifer object
dtc = DecisionTreeClassifier()
# Train Decision Tree Classifer
dtc.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = dtc.predict(X_test)
# model Evaluation
acc_dtc = accuracy_score(y_test, y_pred)
acc_dtc

0.9950769230769231

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3147
        True       0.91      0.94      0.92       103

    accuracy                           1.00      3250
   macro avg       0.95      0.97      0.96      3250
weighted avg       1.00      1.00      1.00      3250



In [ ]:
from sklearn.metrics import recall_score

In [ ]:
tpr_dtc = recall_score(y_test, y_pred)
tpr_dtc

0.941747572815534

In [ ]:
from sklearn.tree import DecisionTreeRegressor
# build model
dtr = DecisionTreeRegressor()
# fit classifiers
dtr.fit(X_train, y_train)
# Prediction
y_pred = dtr.predict(X_test)

# model Evaluation
acc_dtr = accuracy_score(y_test, y_pred)
acc_dtr

0.9950769230769231

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3147
        True       0.91      0.93      0.92       103

    accuracy                           1.00      3250
   macro avg       0.96      0.96      0.96      3250
weighted avg       1.00      1.00      1.00      3250



In [ ]:
tpr_dtr = recall_score(y_test, y_pred)
tpr_dtr

0.9320388349514563

In [ ]:
# The KNN Classifier
from sklearn.neighbors import KNeighborsClassifier
# build model
knn_model = KNeighborsClassifier()
# fit classifiers
knn_model.fit(X_train, y_train)
# Prediction
y_pred = knn_model.predict(X_test)

# model Evaluation
acc_knn = accuracy_score(y_test, y_pred)
acc_knn

0.9876923076923076

In [ ]:
tpr_knn = recall_score(y_test, y_pred)
tpr_knn

0.7378640776699029

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      3147
        True       0.85      0.74      0.79       103

    accuracy                           0.99      3250
   macro avg       0.92      0.87      0.89      3250
weighted avg       0.99      0.99      0.99      3250



In [ ]:
# the SVM Classifier
from sklearn import svm
# build model
svm_model = svm.SVC()
# fit classifiers
svm_model.fit(X_train, y_train)
# Prediction
y_pred = svm_model.predict(X_test)
# model Evaluation
acc_svm = accuracy_score(y_test, y_pred)
acc_svm

0.9683076923076923

In [ ]:
tpr_svm = recall_score(y_test, y_pred)
tpr_svm

0.0

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98      3147
        True       0.00      0.00      0.00       103

    accuracy                           0.97      3250
   macro avg       0.48      0.50      0.49      3250
weighted avg       0.94      0.97      0.95      3250



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier()
# fit classifiers
mlp_model.fit(X_train, y_train)
# Prediction
y_pred = mlp_model.predict(X_test)
# model Evaluation
acc_mlp = accuracy_score(y_test, y_pred)
acc_mlp

0.9686153846153847

In [ ]:
tpr_mlp = recall_score(y_test, y_pred)
tpr_mlp

0.019417475728155338

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98      3147
        True       0.67      0.02      0.04       103

    accuracy                           0.97      3250
   macro avg       0.82      0.51      0.51      3250
weighted avg       0.96      0.97      0.95      3250



In [ ]:
# Linear Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# build model
lda = LinearDiscriminantAnalysis()
# fit classifiers
lda.fit(X_train, y_train)
# Prediction
y_pred = lda.predict(X_test)
# model Evaluation
acc_lda = accuracy_score(y_test, y_pred)
acc_lda

0.9843076923076923

In [ ]:
tpr_lda = recall_score(y_test, y_pred)
tpr_lda

0.7766990291262136

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99      3147
        True       0.74      0.78      0.76       103

    accuracy                           0.98      3250
   macro avg       0.87      0.88      0.88      3250
weighted avg       0.98      0.98      0.98      3250



In [ ]:
# The Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
# build model
log_model = LogisticRegression()
# fit classifiers
log_model.fit(X_train, y_train)
# Prediction
y_pred = log_model.predict(X_test)

# model Evaluation
acc_log = accuracy_score(y_test, y_pred)
acc_log

0.968

In [ ]:
tpr_log = recall_score(y_test, y_pred)
tpr_log

0.2912621359223301

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.98      0.99      0.98      3147
        True       0.49      0.29      0.37       103

    accuracy                           0.97      3250
   macro avg       0.73      0.64      0.67      3250
weighted avg       0.96      0.97      0.96      3250



In [ ]:
# The Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
# build model
rf_model = RandomForestClassifier()
# Fitting the classifier
rf_model.fit(X_train, y_train)
# Prediction
y_pred = rf_model.predict(X_test)
# model Evaluation
acc_rf = accuracy_score(y_test, y_pred)
acc_rf

0.9923076923076923

In [ ]:
tpr_rf = recall_score(y_test, y_pred)
tpr_rf

0.8349514563106796

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.99      1.00      1.00      3147
        True       0.91      0.83      0.87       103

    accuracy                           0.99      3250
   macro avg       0.95      0.92      0.93      3250
weighted avg       0.99      0.99      0.99      3250



In [ ]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
# build model
gbc = GradientBoostingClassifier()
# Fitting the classifier
gbc.fit(X_train, y_train)
# Prediction
y_pred = gbc.predict(X_test)
# model Evaluation
acc_gbc = accuracy_score(y_test, y_pred)
acc_gbc

0.9963076923076923

In [ ]:
tpr_gbc = recall_score(y_test, y_pred)
tpr_gbc

0.970873786407767

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3147
        True       0.92      0.97      0.94       103

    accuracy                           1.00      3250
   macro avg       0.96      0.98      0.97      3250
weighted avg       1.00      1.00      1.00      3250



In [ ]:
df = pd.DataFrame(np.array([[acc_dtc, tpr_dtc], [acc_dtr, tpr_dtr], [acc_knn, tpr_knn], [acc_svm, tpr_svm], [acc_mlp, tpr_mlp], [acc_lda, tpr_lda], 
                            [acc_log, tpr_log], [acc_rf, tpr_rf], [acc_gbc, tpr_gbc]]),
                  columns=['accuracy', 'TPR'], 
                  index=['Decision Tree Classifier', 'Decision Tree Regressor', 'KNeighbors Classifier', 'Support Vector Machines Classifier', 'Multi-layer Perceptron Classifier', 
                  'Linear Discriminant Analysis', 'Logistic Regression', 'Random Forest Classifier', 'Gradient Boosting'])
df

,accuracy,TPR
Decision Tree Classifier,0.995077,0.941748
Decision Tree Regressor,0.995077,0.932039
KNeighbors Classifier,0.987692,0.737864
Support Vector Machines Classifier,0.968308,0.000000
Multi-layer Perceptron Classifier,0.968615,0.019417
Linear Discriminant Analysis,0.984308,0.776699
Logistic Regression,0.968000,0.291262
Random Forest Classifier,0.992308,0.834951
Gradient Boosting,0.996308,0.970874


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_values = {'n_estimators': [100], 'learning_rate': [0.1]}
gbc = GradientBoostingClassifier()
gbc_cv = GridSearchCV(gbc, param_grid=grid_values, cv=10, n_jobs=-1)
gbc_cv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'n_estimators': [100]})

In [ ]:
y_pred = gbc_cv.predict(X_test)
acc_gbc_cv = accuracy_score(y_test, y_pred)
acc_gbc_cv

0.9963076923076923

In [ ]:
y_pred = gbc_cv.predict(X_test)
acc_gbc_cv10 = accuracy_score(y_test, y_pred)
acc_gbc_cv10

0.9963076923076923

In [ ]:
tpr_gbc_cv = recall_score(y_test, y_pred)
tpr_gbc_cv

0.970873786407767

In [ ]:
df = pd.DataFrame(np.array([[acc_gbc, tpr_gbc], [acc_gbc_cv, tpr_gbc_cv]]),
                  columns=['accuracy', 'TPR'], 
                  index=['Gradient Boosting Classifier', 'Gradient Boosting Classifier with CV'])
df

,accuracy,TPR
Gradient Boosting Classifier,0.996308,0.970874
Gradient Boosting Classifier with CV,0.996308,0.970874


In [ ]:
# Random Forest Classifier with CV
grid_values = {'max_features': ["auto"], 'min_samples_split': [2], 'n_estimators': [100]}
rf = RandomForestClassifier()
rf_cv = GridSearchCV(rf, param_grid=grid_values, cv=5,n_jobs=-1)
rf_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_features': ['auto'], 'min_samples_split': [2],
                         'n_estimators': [100]})

In [ ]:
y_pred = rf_cv.predict(X_test)
acc_rf_cv = accuracy_score(y_test, y_pred)
acc_rf_cv

0.9932307692307693

In [ ]:
tpr_rf_cv = recall_score(y_test, y_pred)
tpr_rf_cv

0.8640776699029126

In [ ]:
rf_cv.best_params_

{'max_features': 'sqrt', 'min_samples_split': 50, 'n_estimators': 200}

In [ ]:
df = pd.DataFrame(np.array([[acc_rf, tpr_rf], [acc_rf_cv, tpr_rf_cv]]),
                  columns=['accuracy', 'TPR'], 
                  index=['Random Forest Classifier', 'Random Forest Classifier with CV'])
df

,accuracy,TPR
Random Forest Classifier,0.992308,0.834951
Random Forest Classifier with CV,0.993231,0.864078


In [ ]:
grid_values = {'criterion': ["squared_error"]}
dtr = DecisionTreeRegressor()
dtr_cv = GridSearchCV(dtr, param_grid=grid_values, cv=5,n_jobs=-1)
dtr_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error']})

In [ ]:
y_pred = dtr_cv.predict(X_test)
acc_dtr_cv = accuracy_score(y_test, y_pred)
acc_dtr_cv

0.9953846153846154

In [ ]:
tpr_dtr_cv = recall_score(y_test, y_pred)
tpr_dtr_cv

0.941747572815534

In [ ]:
df = pd.DataFrame(np.array([[acc_dtr, tpr_dtr], [acc_dtr_cv, tpr_dtr_cv]]),
                  columns=['accuracy', 'TPR'], 
                  index=['Decision Tree Regressor', 'Decision Tree Regressor with CV'])
df

,accuracy,TPR
Decision Tree Regressor,0.995077,0.932039
Decision Tree Regressor with CV,0.995385,0.941748
